In [1]:
# Import required libraries
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pytesseract  # OCR for text extraction
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# Define a custom dataset class
class HandwritingDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Simple CNN Model for classification
class DysgraphiaCNN(nn.Module):
    def __init__(self):
        super(DysgraphiaCNN, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Binary classification: Dysgraphic / Non-Dysgraphic
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.fc_layers(x)
        return x

# Load dataset paths and labels (simulate CSV for now)
data_dir = 'dysgraphiaData/'  # Folder containing images
metadata = pd.read_csv('labels.csv')  # CSV with columns: filename, label (0/1)

image_paths = [os.path.join(data_dir, fname) for fname in metadata['filename']]
labels = metadata['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Define transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# DataLoaders
train_dataset = HandwritingDataset(X_train, y_train, transform=transform)
test_dataset = HandwritingDataset(X_test, y_test, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DysgraphiaCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Function for OCR and neurological insights with enhanced preprocessing
def analyze_handwriting(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Enhanced Preprocessing
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    adaptive_thresh = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    kernel = np.ones((2, 2), np.uint8)
    morphed = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.dilate(morphed, kernel, iterations=1)

    # OCR Extraction
    config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(cleaned, config=config)

    print("\nExtracted Text:")
    print(text)

    # Example pattern analysis for neurological conditions (simplified)
    if any(char.isdigit() for char in text) and len(text.split()) < 3:
        print("\nPotential cognitive irregularities detected (e.g., low word count, numeric confusion).")
    elif len(text.strip()) == 0:
        print("\nNo readable text detected — may indicate severe motor issues.")
    else:
        print("\nText structure appears typical.")

# Example usage
sample_image = 'image2.png'
analyze_handwriting(sample_image)  # Run OCR and basic insight analysis

Epoch 1/10, Loss: 2.4819
Epoch 2/10, Loss: 0.7306
Epoch 3/10, Loss: 0.6921
Epoch 4/10, Loss: 0.6897
Epoch 5/10, Loss: 0.6747
Epoch 6/10, Loss: 0.6601
Epoch 7/10, Loss: 0.6385
Epoch 8/10, Loss: 0.5526
Epoch 9/10, Loss: 0.4907
Epoch 10/10, Loss: 0.4332
Test Accuracy: 62.00%

Extracted Text:
We —quide—roun
e—Ayide. oven _fox_ jumped, —
Treaauie bibvrp a :
120 Cay es |
pre recon


Text structure appears typical.
